In [1]:
#from https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('linksResult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "links"
    start_urls = [ 'https://en.wikipedia.org/wiki/Philosophy' ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'linksResult.json'                        # Used for pipeline 2
    }
    
    #Todo: fix parsing for wikipedia
    def parse(self, response):
        for link in response.css('div.mw-parser-output a'):
            linkString = str(link.css('::attr(href)').extract_first())
            if ( (linkString[0:6] == '/wiki/') & (linkString[6:11] != 'File:') ):
                yield {
                    'text': linkString,
                }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-12-16 13:54:39 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:54:39 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 | packaged by conda-forge | (default, Nov 27 2020, 19:31:52) - [GCC 9.3.0], pyOpenSSL 20.0.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.2.1, Platform Linux-4.9.184-linuxkit-x86_64-with-glibc2.10
2020-12-16 13:54:39 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 13:54:39 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-12-16 13:54:39 [py.warnings] WARNING: /opt/conda/lib/python3.8/site-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter 

In [30]:
import pandas as pd
dfjl = pd.read_json('linksResult.jl', lines=True)
dfjl
print(["leaf3", "leaf1", "leaf5"])
for i,j in enumerate(list(dfjl["text"].to_numpy())):
    print(i,j.split("/")[-1])

['leaf3', 'leaf1', 'leaf5']
0 Philosophy_(disambiguation)
1 The_School_of_Athens
2 Raphael
3 Ancient_Greek_philosophy
4 Ancient_Greek_architecture
5 Category:Philosophy
6 Plato
7 Immanuel_Kant
8 Friedrich_Nietzsche
9 Gautama_Buddha
10 Confucius
11 Averroes
12 Category:Branches_of_philosophy
13 Aesthetics
14 Epistemology
15 Ethics
16 Philosophy_of_law
17 Logic
18 Metaphysics
19 Philosophy_of_language
20 Philosophy_of_mind
21 Philosophy_of_science
22 Political_philosophy
23 Social_philosophy
24 Category:Philosophy_by_period
25 Ancient_philosophy
26 Pre-Socratic_philosophy
27 Hellenistic_philosophy
28 Medieval_philosophy
29 Modern_philosophy
30 Early_modern_philosophy
31 Late_modern_philosophy
32 Contemporary_philosophy
33 Category:Philosophical_traditions
34 Analytic_philosophy
35 Logical_positivism
36 Ordinary_language_philosophy
37 Continental_philosophy
38 Existentialism
39 Phenomenology_(philosophy)
40 Pragmatism
41 Philosophical_skepticism
42 African_philosophy
43 Eastern_philosophy

604 Deontology
605 Immanuel_Kant
606 Virtue_ethics
607 Epistemology
608 Dignaga
609 Buddhist_logic
610 Epistemology
611 Knowledge
612 Perception
613 Reason
614 Memory
615 Testimony#Philosophy
616 Truth
617 Belief
618 Justification_(epistemology)
619 Reason
620 Philosophical_skepticism
621 Pre-Socratic_philosophy
622 Pyrrho
623 Pyrrhonism
624 Ren%C3%A9_Descartes
625 David_Hume
626 Empiricism
627 Rationalism
628 A_posteriori_knowledge
629 Philosophy_of_science
630 A_priori_knowledge
631 Logic
632 Mathematics
633 Belief
634 Truth
635 Justification_(epistemology)
636 Gettier_problem
637 Regress_argument
638 Foundationalism
639 Circular_reasoning
640 Coherentism
641 Infinite_regress
642 Infinitism
643 Metaphysics
644 Metaphysics_(Aristotle)
645 Reality
646 Existence
647 Time
648 Object_(philosophy)
649 Property_(philosophy)
650 Causality
651 Mind
652 Human_body
653 Cosmology
654 World
655 Ontology
656 Being
657 Philosophical_realism
658 Idealism
659 Personal_identity
660 Essence
661 Acciden

1085 Hellenistic_philosophy
1086 Aristotelianism
1087 Atomism
1088 Cynicism_(philosophy)
1089 Cyrenaics
1090 Eleatics
1091 Eretrian_school
1092 Epicureanism
1093 Hermeneutics
1094 Ionian_School_(philosophy)
1095 Ephesian_school
1096 Milesian_school
1097 Megarian_school
1098 Neoplatonism
1099 Peripatetic_school
1100 Platonism
1101 Pluralist_school
1102 Pre-Socratic_philosophy
1103 Pyrrhonism
1104 Pythagoreanism
1105 Neopythagoreanism
1106 Sophist
1107 Stoicism
1108 Indian_philosophy
1109 Hindu_philosophy
1110 Samkhya
1111 Nyaya
1112 Vaisheshika
1113 Yoga_Sutras_of_Patanjali
1114 M%C4%ABm%C4%81%E1%B9%83s%C4%81
1115 %C4%80j%C4%ABvika
1116 Aj%C3%B1ana
1117 Charvaka
1118 Jain_philosophy
1119 Anekantavada
1120 Anekantavada#Syādvāda
1121 Buddhist_philosophy
1122 %C5%9A%C5%ABnyat%C4%81
1123 Madhyamaka
1124 Yogachara
1125 Sautr%C4%81ntika
1126 Svatantrika%E2%80%93Prasa%E1%B9%85gika_distinction
1127 Iranian_philosophy
1128 Mazdak#Mazdakism
1129 Mithraism
1130 Zoroastrianism
1131 Zurvanism
1132 M

1559 Group_polarization
1560 Groupshift
1561 Herd_behavior
1562 Holism
1563 Hysterical_contagion
1564 Information_cascade
1565 Invisible_hand
1566 Lynching
1567 Majoritarianism
1568 Ochlocracy
1569 Mass_action_(sociology)
1570 Mass_hysteria
1571 Mass_psychogenic_illness
1572 Milieu_control
1573 Mobbing
1574 Moral_panic
1575 Organization
1576 Peer_pressure
1577 Pluralistic_ignorance
1578 Political_correctness
1579 Pseudoconsensus
1580 Scapegoating
1581 Self-organization
1582 Social_actions
1583 Social_behavior
1584 Social_emotions
1585 Social_exclusion
1586 Social_facilitation
1587 Social_facilitation_in_animals
1588 Social_group
1589 Social_proof
1590 Social_psychology
1591 Sociology
1592 Spontaneous_order
1593 Status_quo
1594 Stigmergy
1595 Swarm_behaviour
1596 System_justification
1597 Viral_phenomenon
1598 Knowledge
1599 Axiom
1600 Tacit_assumption
1601 Conceptual_framework
1602 Epistemology
1603 Outline_of_epistemology
1604 Evidence
1605 Anecdotal_evidence
1606 Scientific_evidence


-3229885564129910096

In [28]:
from neo4j import GraphDatabase, unit_of_work

class Neo4jDatabase:
    # Wrapper around Neo4j database to easily use Cypher in Python
    def __init__(self, uri, user=None, password=None):
        self.max_query_size = 10
        if user and password:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
        else: 
            self.driver = GraphDatabase.driver(uri)
            
    def __del__(self):
        self.driver.close()
    
    @unit_of_work(timeout=5)
    def _get_nodes(self, tx, limit):
        return [dict(record) for record in tx.run(f"MATCH (node) RETURN (node) LIMIT {limit}")]
    
    @unit_of_work(timeout=5)
    def _get_relations(self, tx, limit):
        return [dict(record) for record in tx.run(f"MATCH (root)-[relation]->(leaf) RETURN root, relation, leaf LIMIT {limit}")]
    
    @unit_of_work(timeout=5)
    def _add_nodes(self, tx, query):
        return tx.run(query)

    def get_all_nodes(self, limit=25):
        # Return all nodes in limit
        with self.driver.session() as session:
            return session.read_transaction(self._get_nodes, limit)
        
    def get_all_relations(self, limit=25):
        # Return all nodes in limit
        with self.driver.session() as session:
            return session.read_transaction(self._get_relations, limit)
            
    def add_new_page(self, root, leaves):
        # Add a new root node and create a relationship with all the leaves
        # This function create new leaves if it don't already exist
        list_of_leaves = split_list_in_sublist(leaves)
        for leaves in list_of_leaves:
            query = "merge (%s:WikiPage {link: '%s'})\n" % (root, root)
            for i, leave in enumerate(leaves):
                query += "merge (%s:WikiPage {link: '%s'})\n" % ("n"+str(i), leave)
            for i, leave in enumerate(leaves):
                query += "merge (%s)-[:IsIn]->(%s)\n" % (root, "n"+str(i))
            print(query)
            with self.driver.session() as session:
                session.write_transaction(self._add_nodes, query)
        return "Ok"
    
    def split_list_in_sublist(self, leaves):
        # this function is made to split big list in many list
        _ = 0
        __ = 0
        ___ = []
        while _ < len(leaves):
            _ += 1
            if _ % self.max_query_size == 0:
                ___.append(leaves[__:_])
                __ = _
        if not __ == _:
            ___.append(leaves[__:_])
        return ___

In [31]:
def test_neo4j_wrapper():
    database = Neo4jDatabase("bolt://db:7687")
    #database.add_new_page("leaf2",["leaf3", "leaf9", "Philosophy"])
    database.add_new_page("Philosophy", list(dfjl["text"].to_numpy()))
    print(database.get_all_nodes()) # should print all nodes in database
    print("\n")
    print(database.get_all_relations()) # should print all relations in database
test_neo4j_wrapper()

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Philosophy_(disambiguation)'})
merge (n1:WikiPage {link: '/wiki/The_School_of_Athens'})
merge (n2:WikiPage {link: '/wiki/Raphael'})
merge (n3:WikiPage {link: '/wiki/Ancient_Greek_philosophy'})
merge (n4:WikiPage {link: '/wiki/Ancient_Greek_architecture'})
merge (n5:WikiPage {link: '/wiki/Category:Philosophy'})
merge (n6:WikiPage {link: '/wiki/Plato'})
merge (n7:WikiPage {link: '/wiki/Immanuel_Kant'})
merge (n8:WikiPage {link: '/wiki/Friedrich_Nietzsche'})
merge (n9:WikiPage {link: '/wiki/Gautama_Buddha'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/List_of_unsolved_problems_in_philosophy'})
merge (n1:WikiPage {link: '/wiki/List_of_important_publications_in_philosophy'})
merge (n2:WikiPage {link: '/wiki/List_of_philosophies'})
merge (n3:WikiPage {link: '/wiki/Glossary_of_philosophy'})
merge (n4:WikiPage {link: '/wiki/Lists_of_philosophers'})
merge (n5:WikiPage {link: '/wiki/Philosopher'})
merge (n6:WikiPage {link: '/wiki/Wisdom'})
merge (n7:WikiPage {link: '/wiki/Women_in_philosophy'})
merge (n8:WikiPage {link: '/wiki/Portal:Philosophy'})
merge (n9:WikiPage {link: '/wiki/Template:Philosophy_sidebar'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPa

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Modern_philosophy'})
merge (n1:WikiPage {link: '/wiki/Thales'})
merge (n2:WikiPage {link: '/wiki/Socrates'})
merge (n3:WikiPage {link: '/wiki/Pre-Socratic_philosophy'})
merge (n4:WikiPage {link: '/wiki/Ancient_Greek_philosophy'})
merge (n5:WikiPage {link: '/wiki/Plato'})
merge (n6:WikiPage {link: '/wiki/Platonic_Academy'})
merge (n7:WikiPage {link: '/wiki/Aristotle'})
merge (n8:WikiPage {link: '/wiki/Peripatetic_school'})
merge (n9:WikiPage {link: '/wiki/Cynicism_(philosophy)'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Cyrenaics'})
merge (n

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Islamic_philosophy'})
merge (n1:WikiPage {link: '/wiki/Avicenna'})
merge (n2:WikiPage {link: '/wiki/Islamic_Golden_Age'})
merge (n3:WikiPage {link: '/wiki/Islamic_philosophy'})
merge (n4:WikiPage {link: '/wiki/Islamic_culture'})
merge (n5:WikiPage {link: '/wiki/Arabic'})
merge (n6:WikiPage {link: '/wiki/Muslim_conquests'})
merge (n7:WikiPage {link: '/wiki/Translation_Movement'})
merge (n8:WikiPage {link: '/wiki/Early_Islamic_philosophy'})
merge (n9:WikiPage {link: '/wiki/Islamic_Golden_Age'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Kalam'}

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Classical_Chinese'})
merge (n1:WikiPage {link: '/wiki/Pali'})
merge (n2:WikiPage {link: '/wiki/Transculturalism'})
merge (n3:WikiPage {link: '/wiki/Mahayana'})
merge (n4:WikiPage {link: '/wiki/Theravada'})
merge (n5:WikiPage {link: '/wiki/Southeast_Asia'})
merge (n6:WikiPage {link: '/wiki/Sri_Lanka'})
merge (n7:WikiPage {link: '/wiki/Burma'})
merge (n8:WikiPage {link: '/wiki/Thailand'})
merge (n9:WikiPage {link: '/wiki/Avidy%C4%81_(Buddhism)'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Dukkha'})
merge (n1:WikiPage {link: '/wiki/Buddhist_medi

...

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Attentional_bias'})
merge (n1:WikiPage {link: '/wiki/Attitude_polarization'})
merge (n2:WikiPage {link: '/wiki/Belief_bias'})
merge (n3:WikiPage {link: '/wiki/Cognitive_bias'})
merge (n4:WikiPage {link: '/wiki/List_of_cognitive_biases'})
merge (n5:WikiPage {link: '/wiki/Collective_narcissism'})
merge (n6:WikiPage {link: '/wiki/Confirmation_bias'})
merge (n7:WikiPage {link: '/wiki/Congruence_bias'})
merge (n8:WikiPage {link: '/wiki/Cryptomnesia'})
merge (n9:WikiPage {link: '/wiki/Cultural_bias'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Ethn

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Liminality'})
merge (n1:WikiPage {link: '/wiki/Liturgy'})
merge (n2:WikiPage {link: '/wiki/Marriage'})
merge (n3:WikiPage {link: '/wiki/Myth_and_ritual'})
merge (n4:WikiPage {link: '/wiki/Oath'})
merge (n5:WikiPage {link: '/wiki/Pilgrimage'})
merge (n6:WikiPage {link: '/wiki/Play_(activity)'})
merge (n7:WikiPage {link: '/wiki/Rite_of_passage'})
merge (n8:WikiPage {link: '/wiki/Secular_coming-of-age_ceremony'})
merge (n9:WikiPage {link: '/wiki/Ritual'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Social_class'})
merge (n1:WikiPage {link: '/wiki

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Cosmology'})
merge (n1:WikiPage {link: '/wiki/Religious_cosmology'})
merge (n2:WikiPage {link: '/wiki/Creation_myth'})
merge (n3:WikiPage {link: '/wiki/Deity'})
merge (n4:WikiPage {link: '/wiki/Existence_of_God'})
merge (n5:WikiPage {link: '/wiki/Destiny'})
merge (n6:WikiPage {link: '/wiki/Eschatology'})
merge (n7:WikiPage {link: '/wiki/Everything'})
merge (n8:WikiPage {link: '/wiki/Nothing'})
merge (n9:WikiPage {link: '/wiki/Evolution'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Existence'})
merge (n1:WikiPage {link: '/wiki/Fiction'})
merge

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Vice'})
merge (n1:WikiPage {link: '/wiki/Work_of_art'})
merge (n2:WikiPage {link: '/wiki/Wrongdoing'})
merge (n3:WikiPage {link: '/wiki/Attitude_(psychology)'})
merge (n4:WikiPage {link: '/wiki/Optimism'})
merge (n5:WikiPage {link: '/wiki/Pessimism'})
merge (n6:WikiPage {link: '/wiki/Recluse'})
merge (n7:WikiPage {link: '/wiki/Weltschmerz'})
merge (n8:WikiPage {link: '/wiki/Ideology'})
merge (n9:WikiPage {link: '/wiki/Authoritarianism'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Anarchism'})
merge (n1:WikiPage {link: '/wiki/Capitalism'})
mer

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Modernism'})
merge (n1:WikiPage {link: '/wiki/Postmodernism'})
merge (n2:WikiPage {link: '/wiki/Monism'})
merge (n3:WikiPage {link: '/wiki/Natural_Law'})
merge (n4:WikiPage {link: '/wiki/School_of_Naturalists'})
merge (n5:WikiPage {link: '/wiki/Naturalism_(philosophy)'})
merge (n6:WikiPage {link: '/wiki/Nihilism'})
merge (n7:WikiPage {link: '/wiki/Peripatetic_school'})
merge (n8:WikiPage {link: '/wiki/Phenomenology_(philosophy)'})
merge (n9:WikiPage {link: '/wiki/Platonism'})
merge (Philosophy)-[:IsIn]->(n0)
merge (Philosophy)-[:IsIn]->(n1)
merge (Philosophy)-[:IsIn]->(n2)
merge (Philosophy)-[:IsIn]->(n3)
merge (Philosophy)-[:IsIn]->(n4)
merge (Philosophy)-[:IsIn]->(n5)
merge (Philosophy)-[:IsIn]->(n6)
merge (Philosophy)-[:IsIn]->(n7)
merge (Philosophy)-[:IsIn]->(n8)
merge (Philosophy)-[:IsIn]->(n9)

merge (Philosophy:WikiPage {link: 'Philosophy'})
merge (n0:WikiPage {link: '/wiki/Neoplatonism'})
merge (n